In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset

In [47]:
def prepared_data(file_path):
    
    # 读入训练集和验证集
    train_data = np.load(file_path + '/train.npz')
    valid_data = np.load(file_path + '/valid.npz')
    
    # 从NPZ文件中提取数据
    # guest是稠密特征（数值型），host是稀疏特征（类别型）
    trn_dense = train_data['guest']  # 训练集稠密特征
    trn_sparse = train_data['host']  # 训练集稀疏特征
    trn_y = train_data['labels']     # 训练集标签
    
    val_dense = valid_data['guest']  # 验证集稠密特征
    val_sparse = valid_data['host']  # 验证集稀疏特征
    val_y = valid_data['labels']     # 验证集标签
    
    # 构建特征列信息
    # 稠密特征列（数值型特征）
    dense_features = [{'name': f'I{i}', 'type': 'dense'} for i in range(trn_dense.shape[1])]
    # 稀疏特征列（类别型特征）
    sparse_features = [{'name': f'C{i}', 'type': 'sparse', 'feat_num': int(trn_sparse[:, i].max()) + 1, 'embed_dim': 8} 
                      for i in range(trn_sparse.shape[1])]
    
    # 特征列信息
    fea_col = [dense_features, sparse_features]
    
    # 返回特征列信息和数据集
    return fea_col, (trn_dense, trn_sparse, trn_y), (val_dense, val_sparse, val_y)


fea_cols, (trn_dense, trn_sparse, trn_y), (val_dense, val_sparse, val_y) = prepared_data("./data/criteo")

# 把数据构建成数据管道
# 创建训练集的TensorDataset
dl_train_dataset = TensorDataset(
    torch.tensor(trn_dense).float(), 
    torch.tensor(trn_sparse).long(), 
    torch.tensor(trn_y).float()
)
# 创建验证集的TensorDataset
dl_val_dataset = TensorDataset(
    torch.tensor(val_dense).float(), 
    torch.tensor(val_sparse).long(), 
    torch.tensor(val_y).float()
)

# 创建训练集的DataLoader
dl_train = DataLoader(dl_train_dataset, shuffle=True, batch_size=128)
# 创建验证集的DataLoader
dl_val = DataLoader(dl_val_dataset, shuffle=True, batch_size=128)
tensor_x = torch.from_numpy(trn_dense[0])
tensor_y = torch.from_numpy(trn_y[[0]])
# print(tensor_x)
# print(tensor_y)


class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()
        self.fc1 = nn.Linear(13, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = model()
params = model.parameters()
optimizer = optim.Adam(params, lr=0.001)
loss_fn = nn.BCEWithLogitsLoss()
# print(params)

output = model(tensor_x)
output.retain_grad()
loss = loss_fn(output, tensor_y)
optimizer.zero_grad()    
loss.backward()
print("output.grad:")
print(output.grad)

optimizer.step()

model.parameters
# for name, parms in model.named_parameters():	
#     print('-->name:', name)
#     print('-->para:', parms)
#     print('-->grad_requirs:',parms.requires_grad)
#     print('-->grad_value:',parms.grad)
#     print("===")

output.grad:
tensor([0.5470])


<bound method Module.parameters of model(
  (fc1): Linear(in_features=13, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=1, bias=True)
)>

In [ ]:
####测试backward
def model_test(x,w1,w2,b):
    y = w1*x**2+w2*x+b
    return y

test_params = torch.tensor([1.0, 0.0, 1.0], requires_grad=True)

def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

# outputs = torch.tensor([1.0], requires_grad=True)

outputs = model_test(2, *test_params)
outputs.retain_grad()
loss = loss_fn(outputs, 9)

loss.backward()
print(outputs.grad)



tensor(-8.)


In [ ]:
##例子
params = torch.tensor([1.0,1.0,1.0],requires_grad=True)     #初始化3个参数b1,w1,b2
def model_test(a,b,d,b1,w1,b2):
    y = (a+b+b1)*d*w1+b2
    return y

print(params[0].is_leaf)
print(params[1].is_leaf)
print(params[2].is_leaf)


False
False
False
True
